
### Make the stuff do the thing

In [ ]:
%tensorflow_version 1.x
!pip install gpt-2-simple 
import json
import os
import numpy as np
import tensorflow as tf
import gpt_2_simple as gpt2

model_name = "345M" # The GPT-2 model we're using

gpt2.download_gpt2(model_name=model_name) # Download the model

     |████████████████████████████████| 675kB 7.8MB/s 
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7-cp36-none-any.whl size=23557 sha256=6b6382bd086d8323057ebf0372a8914b424df4781b0eeb7daaf734bd985f5f3b
  Stored in directory: /root/.cache/pip/wheels/28/7f/89/1253cc7ae7fd1cdf130fa146ab17314fd2a5a6d48ccf21dec5
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609187 sha256=ee55b87250426ea14d1a1ef16ee5b422cba2f4c8e5327f23acdad355e2ae6a4e
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built gpt-2-simple regex
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 275Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 102Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 225Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:12, 118Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 290Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 103Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 168Mit/s]                                                       


### Finetune Model
Training the model on a custom dataset (50 positive and 50 negative samples from the stanford sentiment analysis dataset)

In [ ]:
!wget https://github.com/spronkoid/GPT2-sentiment-analysis/blob/master/sentiment50.txt
file_name = "sentiment50.txt" # where the dataset is located

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              steps=2)   # steps is max number of training steps


--2019-12-10 00:23:01--  https://github.com/spronkoid/GPT2-sentiment-analysis/blob/master/sentiment50.txt
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘sentiment50.txt’

sentiment50.txt         [ <=>                ] 235.61K  --.-KB/s    in 0.05s   

2019-12-10 00:23:02 (4.52 MB/s) - ‘sentiment50.txt’ saved [241264]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/345M/model.ckpt
INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.79it/s]


dataset has 84338 tokens
Training...
[1 | 91.04] loss=2.12 avg=2.12
[2 | 171.33] loss=1.96 avg=2.04
Saving checkpoint/run1/model-2


### Cool Stuff

In [ ]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

### Use It
In the prompt, prefix it with // and suffix it with || to get the model to respond with positive or negative


In [ ]:
!git clone https://github.com/openai/gpt-2.git
import os
os.chdir("gpt-2/src/")
import tensorflow as tf
import model, sample, encoder
os.chdir('../../')


Cloning into 'gpt-2'...
remote: Enumerating objects: 221, done.
remote: Total 221 (delta 0), reused 0 (delta 0), pack-reused 221
Receiving objects: 100% (221/221), 4.37 MiB | 20.64 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [ ]:
!ls
interact_model(
    'run1',
    None,
    1,
    1,
    2,
    1,
    0,
    './checkpoint'
)

checkpoint  gpt-2  models  sample_data	sentiment50.txt




Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from ./checkpoint/run1/model-2
Model prompt >>> // no ||
======================================== SAMPLE 1 ========================================
 is_


KeyboardInterrupt: ignored

Created by [@spronkoid](https://github.com/spronkoid)